In [43]:
import subprocess
import json
import os
from bit import PrivateKeyTestnet
import web3
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
from dotenv import load_dotenv
load_dotenv()

from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [44]:
from constants import *
mnemonic = os.getenv('MNEMONIC')

In [126]:
def derive_wallets(coin):
    mnemonic = os.getenv('MNEMONIC')

    #ln does not work for me, replaced with actual php instead
    command = f'php hd-wallet-derive\hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --format=json --coin={coin}'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    return keys

def priv_key_to_account(coin, priv_key):
    if (coin == 'eth'):
        return web3.eth.Account.privateKeyToAccount(priv_key)
    elif (coin == 'btc-test'):
        return PrivateKeyTestnet(priv_key)
    else:
        return None

def create_tx(coin , priv_key, to, amount):
    account = priv_key_to_account(coin, priv_key)
    if (coin == 'eth'):
        return {
            'to': to,
            'from': account.address,
            'value': amount,
            'gas': w3.eth.estimateGas({
                "from": account.address,
                "to": to,
                "value": amount
            }),
            'gasPrice': w3.eth.gasPrice,
            'nonce': w3.eth.getTransactionCount(account.address)
        }
    elif (coin == 'btc-test'):
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])
    else:
        return None

def send_tx(coin, priv_key, to, amount):
    
    account = priv_key_to_account(coin, priv_key)
    raw_tx = create_tx(coin, priv_key, to, amount)

    if (coin == 'eth'):
        signed = account.sign_transaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    elif (coin == 'btc-test'):
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)
    else:
        return None

In [128]:
coin_list = [BTC, ETH, BTCTEST]
coins = {}
for coin in coin_list:
    coins[coin] = derive_wallets(coin)

{'btc': [{'path': "m/44'/0'/0'/0/0",
   'address': '17YmZR7TtK8LYR4KSdFCv7cWmw3sZVJPqA',
   'privkey': 'L4TmignWjXV6beSG85H32TRTa8jpLa4vusa5AKR3DB4yqoCowguo',
   'pubkey': '03976fd2fb02c9d5e5d9fa58f2ff18fd8fc842ecbc5c34c8a906a42f20b289e019'},
  {'path': "m/44'/0'/0'/0/1",
   'address': '1CEgpahg7m3ocmkPRpBenozAuURSnMpyJX',
   'privkey': 'KxKz5693fNEx4ibpMa651eGMpLFhBqnBLh4hGQ2oFagTnNejuTWy',
   'pubkey': '02cb9769f54807cb8cb88934a0961a7722e45192fded77784166bc042c76b04fbd'},
  {'path': "m/44'/0'/0'/0/2",
   'address': '1KwDNyU42rjTPPDPJJ9vhbqEDHcEBqS6Ap',
   'privkey': 'L2zjEnMaPbFxktd2L6ShmNTtEegPHbEV6pZUKFSTeDyovJAkx3qW',
   'pubkey': '03727b38b6e930ffaecad7ad5bae0bf8e4d5f4310eacde09c03219872b85d83bae'},
  {'path': "m/44'/0'/0'/0/3",
   'address': '1K9AQJWmZLTKQvJZLzQaJq6nqzdbxQvL6Q',
   'privkey': 'KyH9JaJGU4RhDWwRstVvVEQjQPUfF43fT7NcZD3qpTYar5ihMC1Z',
   'pubkey': '0241e0449e7ec948b7bf4e17ccee3c9c0d5a1bf412edef24002023916568d1ca63'},
  {'path': "m/44'/0'/0'/0/4",
   'address': '1PkL

In [129]:
coins[BTCTEST]

[{'path': "m/44'/1'/0'/0/0",
  'address': 'mrEpvqTHeU52Aj5SRsux33V4xwqEmWcbAW',
  'privkey': 'cTvT1LAHD9X3BLiKRJbrtCkErMeRaJwpDrNtZCAQw9MGWycJ2pXf',
  'pubkey': '02c15d8c1d85ca5db116fc6ad7e8c48c3a2be7b3f9877d02069322a52b654dfff7'},
 {'path': "m/44'/1'/0'/0/1",
  'address': 'mpJbDpsFBtu8wDJDsLuHBcGbnf3T2BhEQL',
  'privkey': 'cQbwVLXcXjH75vAnTr4CjFriqmn7yXPzuCxvvMeFjJ91ywES8rM1',
  'pubkey': '03fc68ca7877ee3caeb3febcb43b1d563f39fbbbab333235a758cf9389d43a905c'},
 {'path': "m/44'/1'/0'/0/2",
  'address': 'miRZq1aZpGM4aztZdAPezmUEtNP8f6fwQD',
  'privkey': 'cVt5KTfpsrnKUiMYeqeVequYr5jXby5xnoToMtPzky84rk54UHxR',
  'pubkey': '03a46b7afa200f696edd20e1f93a0cd0ad9b1f484f8062ef0bc5bb420404c1f6dc'},
 {'path': "m/44'/1'/0'/0/3",
  'address': 'mrnKe5nJHmZjcJURvHoa5ptiPTAuRoTzwg',
  'privkey': 'cT6R7RN89QT13LZKVagCD6uz2Y8ZBoe18m8E3HkB9dFA1LFSShCc',
  'pubkey': '030cf5cf0885a4f67688d2943ab996b24d5f30e0bc31de25ff5f969edac8f170b9'},
 {'path': "m/44'/1'/0'/0/4",
  'address': 'msQ2is4R3wNuHpA2AVQNYgK79Kiaf

In [131]:
create_tx(BTCTEST, coins[BTCTEST][0]['privkey'], coins[BTCTEST][1]['address'], 0.0001)

'{"unspents":[{"amount":479757,"confirmations":0,"script":"76a914759c945d8ddf56b911998ed6dd57d4ffea797f0388ac","txid":"2695c01e64c719fd11a96e5c7590774bb25dd497f61279cddb9890660444184f","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mpJbDpsFBtu8wDJDsLuHBcGbnf3T2BhEQL",10000],["mrEpvqTHeU52Aj5SRsux33V4xwqEmWcbAW",392465]]}'

In [132]:
raw_tx = create_tx(BTCTEST, coins[BTCTEST][0]['privkey'], coins[BTCTEST][1]['address'], 0.0001)
# raw_tx = create_tx(ETH, priv_key_to_account(ETH, coins[ETH][0]['privkey']), coins[ETH][1]['address'], 1)
raw_tx
# dir(w3.eth)

'{"unspents":[{"amount":479757,"confirmations":0,"script":"76a914759c945d8ddf56b911998ed6dd57d4ffea797f0388ac","txid":"2695c01e64c719fd11a96e5c7590774bb25dd497f61279cddb9890660444184f","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mpJbDpsFBtu8wDJDsLuHBcGbnf3T2BhEQL",10000],["mrEpvqTHeU52Aj5SRsux33V4xwqEmWcbAW",392465]]}'

In [133]:
from bit.network import NetworkAPI

In [134]:
account = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
# signed = account.sign_transaction(raw_tx)
# account.sign_transaction(raw_tx)
account.sign_transaction(raw_tx)

'01000000014f184404669098dbcd7912f697d45db24b7790755c6ea911fd19c7641ec09526010000006a47304402200544d62f67bddccfd48e8548129129b40aa9ae655336cf28793408071d946c40022004988ba74a29d34a74ed2ec8b7823dcdd4b2c1317eb743a7c178a24fd4218677012102c15d8c1d85ca5db116fc6ad7e8c48c3a2be7b3f9877d02069322a52b654dfff7ffffffff0210270000000000001976a91460628cbc1e2cb0d8fd74571264c6cea0bc68464e88ac11fd0500000000001976a914759c945d8ddf56b911998ed6dd57d4ffea797f0388ac00000000'

In [103]:
w3.eth.sendRawTransaction(signed.rawTransaction)

HexBytes('0x46a1b3a3fb18c81211fd074b6edcde722c67a5a9657e854ec391a9e73c0f4e59')